# **Proper Orthogonal Decomposition (POD)**
This lab focuses on performing POD on the parametric system we saw together in the previous lab.

Once again, we need _a lot_ of FOM simulations this time. Let us import ``gedim``!

In [ ]:
!git clone https://github.com/fvicini/CppToPython.git
%cd CppToPython

In [ ]:
!git submodule init
!git submodule update

In [ ]:
!mkdir -p externals
%cd externals
!cmake -DINSTALL_VTK=OFF -DINSTALL_LAPACK=OFF ../gedim/3rd_party_libraries
!make -j4
%cd ..

In [ ]:
!mkdir -p release
%cd release 
!cmake -DCMAKE_PREFIX_PATH="/content/CppToPython/externals/Main_Install/eigen3;/content/CppToPython/externals/Main_Install/triangle;/content/CppToPython/externals/Main_Install/tetgen;/content/CppToPython/externals/Main_Install/googletest" ../
!make -j4 GeDiM4Py
%cd ..

In [ ]:
import numpy as np
import GeDiM4Py as gedim

In [ ]:
lib = gedim.ImportLibrary("./release/GeDiM4Py.so")

config = { 'GeometricTolerance': 1.0e-8 }
gedim.Initialize(config, lib)

## The parametric version of the heat conductivity equation

Solve the following equation on square ${\Omega} = (-1, +1) \times (-1, +1)$

$$
\begin{cases}
\nabla \cdot (k_{\mu} \nabla u) = 0 & \text{in } \Omega\\
k_{\mu} \nabla u \cdot n_1 = \mu_2 & \text{in } \Gamma_{base}\\
u = 0 & \text{in } \Gamma_{top}\\
k_{\mu} \nabla u \cdot n_2 = 0 & \text{otherwise} 
\end{cases}
$$

where $k_{\mu} = \mu_1$ if $x^2 + y^2 \leq R^2$ and $k = 1$ otherwise. 
The parametric space is $\mathcal P = [0.1, 10] \times [-1,1]$.

<img src="https://drive.google.com/uc?id=1j98eKPtRy8IqsLMKkue2dINRy6yNS20j"
 style="float:center;width:50px;height:50px;" align="center">


The parameter $\boldsymbol \mu \in \mathcal P$ is physical and changes the features of the flow: 

1. $\mu_1$ the conductivity in $\Omega_1$;
2. $\mu_2$ describes the heat flux in the bottom part of the boundary.

First thing: we define two subdomains $\Omega_1$ and $\Omega_2$, such that
1. $\Omega_1$ is a disk in the origin with radius $r_0=0.5$, and
2. $\Omega_2=\Omega/\ \overline{\Omega_1}$.
3. $\Gamma_{base}$ to define where we will change the heat flux.


In [ ]:
def Heat_R():
	return 0.5


def Omega1(numPoints, points):
	matPoints = gedim.make_nd_matrix(points, (3, numPoints), np.double)
	values = np.ones(numPoints)
	for p in range(0, numPoints):
		if (matPoints[0,p] * matPoints[0,p] + matPoints[1,p] * matPoints[1,p]) > (Heat_R() * Heat_R() + 1.0e-16):
			values[p] = 0.
	return values.ctypes.data

def Omega2(numPoints, points):
	matPoints = gedim.make_nd_matrix(points, (3, numPoints), np.double)
	values = np.ones(numPoints)
	for p in range(0, numPoints):
		if (matPoints[0,p] * matPoints[0,p] + matPoints[1,p] * matPoints[1,p]) <= (Heat_R() * Heat_R() + 1.0e-16):
			values[p] = 0. 
	return values.ctypes.data

def Gamma_base(numPoints, points):
	values = np.ones(numPoints)
	return values.ctypes.data

##### needed for the inner product #####

def Domain(numPoints, points):
	matPoints = gedim.make_nd_matrix(points, (3, numPoints), np.double)
	values = np.ones(numPoints)
	return values.ctypes.data	

**Goal**: build the ROM space where many simulations for several parameters can be performed in a smaller amount of time. 

**Strategy**: $w(\boldsymbol  \mu)  \xrightarrow[]{\text{FOM} (\dim = \mathcal N)} w^{\mathcal N}(\boldsymbol \mu)
\xrightarrow[\lvert \lvert {w(\boldsymbol \mu) - w^\mathcal{N}(\boldsymbol  \mu)\rvert \rvert } \rightarrow 0]{\text{ROM } (\dim N)} w_N(\boldsymbol  \mu)$.

The goal can be reached by means of several techniques. 

Today we will focus on POD.

POD is an _explore_ and _compress_ algorithm based on two different stages:
1. we _explore_ the information related to the solution varying with respect to $\boldsymbol \mu$ (_snapshots_) in a finite dimensional set $\mathcal P_{train} \subset \mathcal P$.
2. We compress the redundant information and retain only the most significant "linear directions", building a linear subspace $\mathbb V_N \subset \mathbb V^{\mathcal N}$ of dimension $N \ll \mathcal N$.

Building the space and store the $\boldsymbol \mu-$independent quantities is the so called _offline phase_ (possibly costly).

Once the space is built, a fast _online phase_ occurs, where I can compute **many solutions in real-time**.

How is it possible? By means of the affine decomposition! Indeed we know that our system can be written as

$$
\sum_{i=1}^{q_a} \theta_i^a(\boldsymbol \mu)a_i(u,v) = \sum_{i=0}^{q_f} \theta_i^f(\boldsymbol \mu)f_i(v),
$$

i.e., algebraic-wise

$$
\sum_{i=1}^{q_a} \theta_i^a(\boldsymbol \mu)\mathsf A_i = \sum_{i=0}^{q_f} \theta_i^f(\boldsymbol \mu)\mathsf f_i,
$$
where $\mathsf A_i$ and $\mathsf f_i$ are the assembled matrices and vectors of the system.

Now, let us imagine to have already built the reduced space and have collected the basis functions $\xi_{i} \in \mathbb R^{\mathcal N}$ for $i \in \{1, \dots, N \}$ ($\mathbb V_N = \text{span}\{\xi_i\}_{i=1}^{N} $) in a basis matrix 
$$
\mathbb B = [\xi_1 \cdots \xi_N] \in \mathbb R^{\mathcal N \times N}.
$$ 

It is clear that we can recast the problem in the low-dimensional framework we built, we can pre-and-post multiply the FOM matrices for the basis matrix we have:

$$
\mathsf A_i^N = \mathbb B^T\mathsf A_i\mathbb B \quad \text{ and } \quad  \mathbb B^T\mathsf f_i.  
$$

**Here is where the offline phase ends!!**

**Question time!**: where did it start?


**Let us code the OFFLINE PHASE**

In [ ]:
### order of the discretization ###
order = 1

In [ ]:
%%writefile ImportMesh.csv
InputFolderPath
./Meshes/Mesh3

In [ ]:
[meshInfo, mesh] = gedim.ImportDomainMesh2D(lib)

In [ ]:
gedim.PlotMesh(mesh)

### FEM space (the High Fidelity approximation)

In [ ]:
discreteSpace = { 'Order': order, 'Type': 1, 'BoundaryConditionsType': [1, 3, 3, 2] }
[problemData, dofs, strongs] = gedim.Discretize(discreteSpace, lib)

In [ ]:
gedim.PlotDofs(mesh, dofs, strongs)

### Assemble linear system exploting affinity
We define everything that is parameter independent, i.e.
$a_1(u,v)$, $a_2(u,v)$ and $f(v)$.
We need also the matrix related to the scalar product of the problem at hand.


In [ ]:

[stiffness1, stiffnessStrong1] = gedim.AssembleStiffnessMatrix(Omega2, problemData, lib)
[stiffness2, stiffnessStrong2] = gedim.AssembleStiffnessMatrix(Omega1, problemData, lib)
	
weakTerm_down1 = gedim.AssembleWeakTerm(Gamma_base, 1, problemData, lib)

#### inner product  
# ||u||^2 + ||grad(u)||^2

# [reaction, reactionStrong] = gedim.AssembleReactionMatrix(Domain, problemData, lib)

inner_product = stiffness1 + stiffness2  ######## semi-norm (equivalent)





We here define the finite parametric space $\mathcal P_{train}$, with random uniform distributed realization of $\boldsymbol \mu$.
The cardinality of $\mathcal P_{train}$ is set to 100 and we call it $M$.

In [ ]:
### define the training set

snapshot_num = 100 # (M)
mu1_range = [0.1, 10.]
mu2_range = [-1., 1.]
P = np.array([mu1_range, mu2_range])

training_set = np.random.uniform(low=P[:, 0], high=P[:, 1], size=(snapshot_num, P.shape[0]))


We now need to define the _snapshot matrix_. The snapshot matrix is $\mathbb U \in \mathbb R^{{M} \times {\mathcal N}}$ 

(is it the dimension on the book?)

In [ ]:
#### snapshot matrix creation
thetaA1 = 1.
snapshot_matrix = []

tol = 1. - 1e-7  
N_max = 10

for mu in training_set:
  thetaA2 = mu[0]
  thetaf1 = mu[1]

  stiffness = thetaA1*stiffness1 + thetaA2*stiffness2
  weakTerm_down = thetaf1*weakTerm_down1
  
  snapshot = gedim.LUSolver(stiffness, weakTerm_down, lib)
  
  snapshot_matrix.append(np.copy(snapshot))

snapshot_matrix = np.array(snapshot_matrix) 

print(snapshot_matrix.shape)

  

**It's time for the POD**

To build the $N-$dimesional framework we need, we define the correlation snapshot matrix $\mathbf C \in \mathbb R^{M \times M}$ and we solve the eigenvalue problem
$
    \mathbf C \omega_n = \lambda_n \omega_n
$ for $ 1 \leq n \leq M,$ with $\lvert \lvert {\omega_n}\rvert \rvert_{\mathbb V} = 1$. 
Due to the definition of correlation matrix, we can order the all-positive eigenvalues as $\lambda_1 >\dots > \lambda_{M}> 0$ and retain the first $N$ eigenpairs $(\lambda_n, \omega_n)$ for $1 \leq n \leq N$. 

**Question Time**: how can I choose $M$ and $N$?

Looking at the eigenvalues! 
Indeed, defining as  $P_N: \mathbb V \rightarrow \mathbb V_N$ the projector from $\mathbb V$ onto $ {\mathbb V}_N$, the following relation holds:
\begin{equation}
    \sqrt{\frac{1}{M}
    \sum_{i = 1}^{M}  \lvert \lvert {u^{\mathcal N}(\boldsymbol{\mu}_{i}) - P_N(u^{\mathcal N}(\boldsymbol{\mu}_i)\rvert \rvert }_{\mathbb V}^2} = \sqrt{
    \sum_{i = N + 1}^{M}\lambda_m.}
\end{equation}
Namely, a fast decay of the eigenvalue magnitude guaratees a good representation of the high-fidelity solution with a few basis functions.

In [ ]:
### covariance matrix

C = snapshot_matrix @ inner_product @ np.transpose(snapshot_matrix) 

###### shape?? ############

#### ALTERNATIVE:  VM, L, VMt = np.linalg.svd((C))

L_e, VM_e = np.linalg.eig(C)
eigenvalues = []
eigenvectors = []


#### check


for i in range(len(L_e)):
  eig_real = L_e[i].real
  eig_complex = L_e[i].imag
  assert np.isclose(eig_complex, 0.)
  eigenvalues.append(eig_real)
  eigenvectors.append(VM_e[i].real)


total_energy = sum(eigenvalues)
retained_energy_vector = np.cumsum(eigenvalues)
relative_retained_energy = retained_energy_vector/total_energy


if all(flag==False for flag in relative_retained_energy>= tol):
  N = N_max
else:
  N = np.argmax(relative_retained_energy >= tol) + 1

print("The reduced dimension is", N)



We still need to create the basis matrix $\mathbb B$. There are many ways to build the bases.
We propose the following one to guarantee more stability:
$$    
\chi_n =  \sum_{m = 1}^{M} (\omega_n)_m u^{\mathcal N}(\boldsymbol{\mu}_m),  \quad \quad 1 \leq n \leq N,
$$

and $\displaystyle \xi_n = \frac{\chi_n}{\lvert \lvert \chi_n \rvert \rvert }_{\mathbb V}$.

In [ ]:
# Create the basis function matrix
basis_functions = []
for n in range(N):
  eigenvector =  eigenvectors[n]
  
  # basis = (1/np.sqrt(snapshot_num))*np.transpose(snapshot_matrix)@eigenvector  (This is the one of the book!!)
  
  basis = np.transpose(snapshot_matrix)@eigenvector
  norm = np.sqrt(np.transpose(basis) @ inner_product @ basis) 
  
  basis /= norm
  basis_functions.append(np.copy(basis))

basis_functions = np.transpose(np.array(basis_functions))


The offline stage ends once the **reduced operators** are built, i.e. $\mathsf A_1^{N}$, $\mathsf A_2^{N}$ and $\mathsf f_1^{N}$ thanks to the basis matrix projections.

In [ ]:
########## ASSEMBLE THE LINEAR SYSTEM ##### STILL OFFLINE
reduced_stiff1 = np.transpose(basis_functions) @ stiffness1 @ basis_functions
reduced_stiff2 = np.transpose(basis_functions) @ stiffness2 @ basis_functions
reduced_f =  np.transpose(basis_functions) @ weakTerm_down1

### shape? ##### 

### **Online Phase: a new parameter!** ###
In the _online phase_ we can use all the pre-assembled quantities to generate a new solution for a new parameter. 



In [ ]:
thetaA2 = 2.
thetaf1 = 0.8

In [ ]:
reduced_rhs = thetaA1*reduced_stiff1 + thetaA2*reduced_stiff2
reduced_lhs = thetaf1*reduced_f

In [ ]:
##### solve ######### 

reduced_solution = np.linalg.solve(reduced_rhs, reduced_lhs)
print(reduced_solution)

In [ ]:
###### plot #######
proj_reduced_solution = basis_functions @ reduced_solution

gedim.PlotSolution(mesh, dofs, strongs, proj_reduced_solution, np.zeros(problemData['NumberStrongs']))

stiffness = thetaA1*stiffness1 + thetaA2*stiffness2
weakTerm_down = thetaf1*weakTerm_down1
  
full_solution = gedim.LUSolver(stiffness, weakTerm_down, lib)



In [ ]:
gedim.PlotSolution(mesh, dofs, strongs, full_solution, np.zeros(problemData['NumberStrongs']))

We can now compute an error analysis over the parametric space, together with a _speed-up_ anaslysis.

The speed-up is an index that evaluated how many ROM solution I can obtain in the time of a FOM simulation.

In [ ]:
### compute error
import time

abs_err = []
rel_err = []
testing_set = np.random.uniform(low=P[:, 0], high=P[:, 1], size=(100, P.shape[0]))
speed_up = []

print("Computing error and speedup analysis")

for mu in testing_set:
  
  thetaA2 = mu[0]
  thetaf1 = mu[1]

  ##### full #####
  stiffness = thetaA1*stiffness1 + thetaA2*stiffness2
  weakTerm_down = thetaf1*weakTerm_down1
  
  start_fom = time.time()
  full_solution = gedim.LUSolver(stiffness, weakTerm_down, lib)
  time_fom = time.time() - start_fom

  #### reduced #####

  reduced_rhs = thetaA1*reduced_stiff1 + thetaA2*reduced_stiff2
  reduced_lhs = thetaf1*reduced_f
  
  start_rom = time.time()
  reduced_solution = np.linalg.solve(reduced_rhs, reduced_lhs)
  time_rom = time.time() - start_rom
  
  speed_up.append(time_fom/time_rom)
  
  proj_reduced_solution = basis_functions@reduced_solution

  ### computing error

  error_function = full_solution - proj_reduced_solution
  error_norm_squared_component = np.transpose(error_function) @ inner_product @ error_function
  absolute_error = np.sqrt(abs(error_norm_squared_component))
  abs_err.append(absolute_error)
  
  full_solution_norm_squared_component = np.transpose(full_solution) @  inner_product @ full_solution
  relative_error = absolute_error/np.sqrt(abs(full_solution_norm_squared_component))
  rel_err.append(relative_error)
  


In [ ]:
print("avarege relative error = ", np.mean(rel_err) )
print("avarege absolute error = ", np.mean(abs_err) )
print("avarege speed_up = ", np.mean(speed_up) )

**Exercise**: solve the same problem but with $u = 3$ on $\Gamma_{top}$.